# Simple Cube Volume Optimization

Each solution or "member" of the population consists of an `x`, `y`, and `z` value.  We are trying to optimize these values to result in the cube with the largest volume.

There are 4 basic operations that go into preparing the next generation: Selection, Crossover, Mutation, and Generation.

## Selection
This one is simple. Just keep the top `n` performers from the previous generation

## Crossover
Perform a "pseudo-merge" of two high-performing solutions to see if the child yields better results

## Mutation
Randomly change some of the values in a high-performing solution to see if it can be improved

## Generation
Create some new random member for the population.  Maybe they'll beat the current top performers.

In [1]:
import random
import operator
import itertools
from pprint import pprint

# How much mutations should alter the value
MUTATION_MODIFIER = 0.05
# Number of solutions in a population
POPULATION_SIZE = 25
# How many iterations should a solution be "on-top" for it to be considered the correct solution?
STATIC_ITERATION_VALUE = 5
# For now, we will just use a set number of iterations.
ITERATION_COUNT = 10

# Rates at which each process occurs.  These should add up to 1.
SELECTION_RATE = .2
CROSSOVER_RATE = .4
MUTATION_RATE = .2
GENERATION_RATE = .2

# Range of values for x, y, and z.   (min, max)
VALUE_RANGE = (0,100)

In [2]:
def fitness(solution):
    return list(itertools.accumulate(solution, operator.mul))[-1]

In [3]:
def crossover(solution1, solution2):
    """Swap 2 of the values"""
    coord_to_cross = random.choice(range(len(solution1)))
    solution1, solution2 = solution1.copy(), solution2.copy()
    solution1[coord_to_cross], solution2[coord_to_cross] = solution2[coord_to_cross], solution1[coord_to_cross]
    return (solution1,solution2)

In [4]:
def create_crossover_pairs(population):
    n_pairs = int(CROSSOVER_RATE * POPULATION_SIZE) // 2
    pairs = []
    for i in range(0, n_pairs*2, 2):
        pairs.append((population[i], population[i+1]))
    # pprint(pairs)
    return pairs
    # return [(population[i],population[i+1]) for i in range(0, n_pairs*2, 2)]

In [5]:
def mutate(solution):
    """Mutate a single coordinate value of the solution.  Either up or down."""
    solution = solution.copy()
    coord_to_mutate = random.choice(range(len(solution)))
    # Could also add random.uniform(0,MUTATION_MODIFER) to allow mutation amount to vary.
    solution[coord_to_mutate] *= 1 + (random.choice([1,-1]) * MUTATION_MODIFIER)
    # Ensure it is in the allowable range though.
    solution[coord_to_mutate] = max(min(solution[coord_to_mutate], VALUE_RANGE[1]), VALUE_RANGE[0])
    return solution

In [6]:
def create_new_solution():
    return [random.uniform(*VALUE_RANGE) for _ in range(3)]

In [7]:
def create_initial_population():
    return [create_new_solution() for _ in range(POPULATION_SIZE)]

In [8]:
def create_next_generation(current_population):
    new_population = []
    # Perform Selection
    n_selections = int(SELECTION_RATE * POPULATION_SIZE)
    selection_candidates = current_population[:n_selections]
    new_population.extend(selection_candidates)
    # Perform Crossover
    for pair in create_crossover_pairs(current_population):
        new_population.extend(crossover(*pair))
    # Perform Mutation
    n_mutations = int(MUTATION_RATE * POPULATION_SIZE)
    new_population.extend([mutate(s) for s in current_population[:n_mutations]])
    # Perform Generation
    n_generations = int(GENERATION_RATE * POPULATION_SIZE)
    new_population.extend([create_new_solution() for _ in range(n_generations)])
    return new_population

In [9]:
# Demonstrate algorithm working with a set number of iterations
population = create_initial_population()
for i in range(ITERATION_COUNT):
    population = sorted(population, key=fitness, reverse=True)
    # print(population)
    print("Current top-performing solution: ")
    print("[{:.4f}, {:.4f}, {:.4f}] with a fitness of {:.4f}".format(*population[0], fitness(population[0])))
    population = create_next_generation(population)


Current top-performing solution: 
[81.7959, 94.8977, 80.3823] with a fitness of 623946.9754
Current top-performing solution: 
[81.7959, 94.8977, 84.4014] with a fitness of 655144.3242
Current top-performing solution: 
[81.7959, 94.8977, 84.4014] with a fitness of 655144.3242
Current top-performing solution: 
[81.7959, 94.8977, 97.2447] with a fitness of 754837.7062
Current top-performing solution: 
[81.7959, 99.6426, 97.2447] with a fitness of 792579.5916
Current top-performing solution: 
[85.8857, 99.6426, 97.2447] with a fitness of 832208.5711
Current top-performing solution: 
[90.1800, 99.6426, 97.2447] with a fitness of 873818.9997
Current top-performing solution: 
[90.1800, 100.0000, 97.2447] with a fitness of 876952.9705
Current top-performing solution: 
[90.1800, 99.6426, 100.0000] with a fitness of 898577.2567
Current top-performing solution: 
[90.1800, 100.0000, 100.0000] with a fitness of 901800.0236


In [10]:
# Demonstrate algorithm working with a convergence check.  
population = sorted(create_initial_population(), key=fitness, reverse=True)
fitness_values = [[population[0], 1]]
n_iterations = 1
while fitness_values[-1][1] < STATIC_ITERATION_VALUE:
    print("Current top-performing solution: ")
    print("[{:.4f}, {:.4f}, {:.4f}] with a fitness of {:.4f}".format(*population[0], fitness(population[0])))
    population = sorted(create_next_generation(population), key=fitness, reverse=True)
    n_iterations += 1
    if fitness_values[-1][0] == population[0]:
        fitness_values[-1][1] += 1
    else:
        fitness_values.append([population[0], 1])
print("~"*15)
print("Optimal solution (possibly) found:")
print("[{:.4f}, {:.4f}, {:.4f}] with a fitness of {:.4f} after {} iterations".format(*population[0], fitness(population[0]), n_iterations))

Current top-performing solution: 
[80.1813, 92.1085, 95.4955] with a fitness of 705270.8809
Current top-performing solution: 
[84.0913, 92.1085, 95.4955] with a fitness of 739662.6455
Current top-performing solution: 
[99.8478, 92.1085, 83.1243] with a fitness of 764480.4002
Current top-performing solution: 
[88.2958, 92.1085, 95.4955] with a fitness of 776645.7778
Current top-performing solution: 
[99.8478, 92.1085, 95.4955] with a fitness of 878256.7000
Current top-performing solution: 
[99.8478, 92.1085, 100.0000] with a fitness of 919683.5234
Current top-performing solution: 
[100.0000, 92.1085, 100.0000] with a fitness of 921085.0055
Current top-performing solution: 
[99.8478, 96.7139, 100.0000] with a fitness of 965667.6996
Current top-performing solution: 
[99.8478, 100.0000, 100.0000] with a fitness of 998478.4444
Current top-performing solution: 
[100.0000, 100.0000, 100.0000] with a fitness of 1000000.0000
Current top-performing solution: 
[100.0000, 100.0000, 100.0000] with 